# Minneapolis Police Incidents 2010 - 2017

url: http://opendata.minneapolismn.gov/

In [1]:
# Import required libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import pickle

%matplotlib inline

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'

### DATA IMPORTING & MERGING

In [3]:
# Read in 8 separate CSV files & create DataFrames
df_dict = {}

file_name_str = 'Police_Incidents_'
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]

for year in years:
    df_dict[year] = pd.read_csv(file_name_str + str(year) + '.csv')

In [4]:
# Concatenate all DFs into 1 DF
df_list = []

for year in sorted(df_dict):
    df_list.append(df_dict[year])
df = pd.concat(df_list, ignore_index=True)

In [5]:
# Look at first few rows of DataFrame
df.head()

,BeginDate,CCN,ControlNbr,Description,ESRI_OID,EnteredDate,GBSID,LastChanged,LastUpdateDate,Lat,...,Neighborhood,OBJECTID,Offense,Precinct,PublicAddress,ReportedDate,Time,UCRCode,X,Y
0,2010-09-20T16:35:00.000Z,MP 2010 291719,2889603,Other Theft,15001.0,2010-09-22T16:39:20.000Z,16434.0,2010-09-23T08:37:46.000Z,2017-03-03T13:40:06.000Z,44.959121,...,VENTURA VILLAGE,NaN,THEFT,3.0,0009XX 24 ST E,2010-09-22T16:40:00.000Z,16:35:00,7,-1.038172e+07,5.615088e+06
1,2010-09-14T03:43:00.000Z,MP 2010 291647,2889606,Burglary Of Business,15002.0,2010-09-22T17:05:24.000Z,19124.0,2010-09-22T20:02:01.000Z,2017-03-03T13:40:06.000Z,44.962751,...,SEWARD,NaN,BURGB,3.0,0022XX Franklin AV E,2010-09-22T17:05:00.000Z,03:43:00,6,-1.037939e+07,5.615659e+06
2,2010-09-22T06:00:00.000Z,MP 2010 291696,2889608,Burglary Of Dwelling,15003.0,2010-09-22T17:08:24.000Z,16592.0,2010-09-23T10:12:50.000Z,2017-03-03T13:40:06.000Z,44.982728,...,MARCY HOLMES,NaN,BURGD,2.0,0013XX 6 ST SE,2010-09-22T17:08:00.000Z,06:00:00,6,-1.037886e+07,5.618803e+06
3,2010-09-21T23:30:00.000Z,MP 2010 291761,2889616,Motor Vehicle Theft,15004.0,2010-09-22T17:40:20.000Z,17645.0,2010-09-23T06:40:19.000Z,2017-03-03T13:40:06.000Z,44.986351,...,MARCY HOLMES,NaN,AUTOTH,2.0,0010XX 8 ST SE,2010-09-22T17:41:00.000Z,23:30:00,8,-1.037915e+07,5.619373e+06
4,2010-09-22T15:00:00.000Z,MP 2010 291752,2889617,Motor Vehicle Theft,15005.0,2010-09-22T17:51:00.000Z,16883.0,2010-09-23T08:38:01.000Z,2017-03-03T13:40:06.000Z,44.951006,...,MIDTOWN PHILLIPS,NaN,AUTOTH,3.0,0028XX 14 AV S,2010-09-22T17:50:00.000Z,15:00:00,8,-1.038110e+07,5.613812e+06


In [6]:
# Create a DataFrame with just the interested columns/variables
df2 = df[['ReportedDate', 'Neighborhood', 'Offense', 'Description', 'Lat', 'Long']]

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172009 entries, 0 to 172008
Data columns (total 6 columns):
ReportedDate    172009 non-null object
Neighborhood    171603 non-null object
Offense         172009 non-null object
Description     172009 non-null object
Lat             172009 non-null float64
Long            172009 non-null float64
dtypes: float64(2), object(4)
memory usage: 7.9+ MB


### DATA CLEANING & MUNGING 

In [8]:
# Check columns for missing data
df2.isnull().sum()

ReportedDate      0
Neighborhood    406
Offense           0
Description       0
Lat               0
Long              0
dtype: int64

In [ ]:
# Drop all rows with any missing data
# df2 = df2.dropna(how='any')

# Declined to drop missing data as only Neighborhood column is missing 
# and variable is not of interest

In [9]:
# Convert ReportedDate column to pandas DateTime object
df2['ReportedDate'] = df2['ReportedDate'].apply(lambda x: pd.to_datetime(x))

# # Create year, month, day_of_week, and time columns
df2['Year'] = df2.ReportedDate.dt.year
df2['Month_Num'] = df2.ReportedDate.dt.month
df2['Day_of_Week_Num'] = df2.ReportedDate.dt.dayofweek + 1
df2['Time'] = df2.ReportedDate.dt.time

# Create mapping for day_of_week
day_of_week = {1:'Mon', 2:'Tues', 3:'Wed', 4:'Thurs', 5:'Fri', 6:'Sat', 7:'Sun'}
month_map = {1: 'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul',
            8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

# Apply mapping to Day_of_Week column
df2['Day_of_Week']= df2.Day_of_Week_Num.map(day_of_week)
df2['Month'] = df2.Month_Num.map(month_map)

# Convert military time to standard time
from datetime import datetime
df2['Time_Standard'] = df2['Time'].apply(lambda d: d.strftime('%I:%M %p'))

In [10]:
# Grab only incidents reported from 2010 - 2017 (2017 dataset has 2018 incidents mixed in)
df2 = df2[df2['Year'] != 2018]

In [11]:
# Save DataFrame 2 to a pickle file
df2.to_pickle('df2.pickle')

In [12]:
# Look at first few rows of df to ensure mapping and cleanings were done correctly
df2.head()

,ReportedDate,Neighborhood,Offense,Description,Lat,Long,Year,Month_Num,Day_of_Week_Num,Time,Day_of_Week,Month,Time_Standard
0,2010-09-22 16:40:00,VENTURA VILLAGE,THEFT,Other Theft,44.959121,-93.260557,2010,9,3,16:40:00,Wed,Sep,04:40 PM
1,2010-09-22 17:05:00,SEWARD,BURGB,Burglary Of Business,44.962751,-93.239666,2010,9,3,17:05:00,Wed,Sep,05:05 PM
2,2010-09-22 17:08:00,MARCY HOLMES,BURGD,Burglary Of Dwelling,44.982728,-93.234905,2010,9,3,17:08:00,Wed,Sep,05:08 PM
3,2010-09-22 17:41:00,MARCY HOLMES,AUTOTH,Motor Vehicle Theft,44.986351,-93.237514,2010,9,3,17:41:00,Wed,Sep,05:41 PM
4,2010-09-22 17:50:00,MIDTOWN PHILLIPS,AUTOTH,Motor Vehicle Theft,44.951006,-93.255022,2010,9,3,17:50:00,Wed,Sep,05:50 PM


### DATA EXPLORATION 

In [13]:
# Which day of the week had the most incidents reported on?
df2['Day_of_Week'].value_counts().head(10)

Mon      27693
Tues     26519
Wed      25302
Fri      24285
Thurs    24198
Sat      20978
Sun      20579
Name: Day_of_Week, dtype: int64

In [14]:
# Which month has the most incidents reported on?
df2['Month'].value_counts()

Aug    17258
Jul    17252
Sep    15924
Oct    15812
Jun    15605
May    14642
Nov    13904
Dec    12902
Apr    12780
Mar    11983
Jan    11735
Feb     9757
Name: Month, dtype: int64

In [15]:
# When are incidents reported the most?
df2['Time_Standard'].value_counts().head(25)

02:00 PM    1180
03:00 PM    1165
01:00 PM    1070
12:00 PM    1030
11:00 AM     997
10:00 AM     974
09:00 AM     937
06:00 PM     848
04:00 PM     832
03:30 PM     791
01:30 PM     789
02:30 PM     781
05:00 PM     765
10:30 AM     730
07:00 PM     726
10:00 PM     692
08:00 PM     691
11:30 AM     686
12:30 PM     685
09:00 PM     667
09:30 AM     662
08:00 AM     645
11:00 PM     597
08:30 AM     550
06:30 PM     541
Name: Time_Standard, dtype: int64

In [16]:
# Top 10 Neighborhood with Police Incidents
df2.Neighborhood.value_counts().head(10)

DOWNTOWN WEST      19146
WHITTIER            6971
JORDAN              5295
LONGFELLOW          4989
MARCY HOLMES        4881
NEAR - NORTH        4759
WILLARD - HAY       4024
LOWRY HILL EAST     3961
HAWTHORNE           3829
LORING PARK         3713
Name: Neighborhood, dtype: int64

In [17]:
# Which year had the most incidents reported?
df2.Year.value_counts().head(10)

2017    22112
2012    21934
2013    21857
2011    21627
2014    21532
2010    20339
2016    20155
2015    19998
Name: Year, dtype: int64

### Top 3 Incidents

In [ ]:
# Explore Top 3 Incident Types by Day of Week
top_3_incidents = df2['Description'].value_counts().head(3)

# Save Series to a pickle file
top_3_incidents.to_pickle('top_3_incidents.pickle')

# Create a Bool variable for whether incident is one of the top 3 types
desc_bool1 = df2['Description'].isin(top_3_incidents.index.tolist())

# # Create a new DF with just top 3 crimes
df_top3 = df2[desc_bool1]

# grouped by COUNT
df_top3_grouped = pd.DataFrame(df_top3.groupby(['Day_of_Week_Num', 'Description'])['Description'].apply(lambda x: x.count()))

# Rename and reset Data Viz DF
df_top3_grouped.rename(columns={'Description':'Count'}, inplace=True)
df_top3_grouped.reset_index(inplace=True)

# Save as a Pickle file
df_top3_grouped.to_pickle('df_top3_grouped.pickle')

### Top 10 Incidents 

In [ ]:
# Explore Top 10 Incident Types by Day of Week
top_10_incidents = df2['Description'].value_counts().head(10)

# Save Series to a pickle file
top_10_incidents.to_pickle('top_10_incidents.pickle')

# Create a Bool variable for whether incident is one of the top ten types
desc_bool2 = df2['Description'].isin(top_10_incidents.index.tolist())

# # Create a new DF with just top 3 crimes
df_top10 = df2[desc_bool2]

# grouped by COUNT
df_top10_grouped = pd.DataFrame(df_top10.groupby(['Day_of_Week_Num', 'Description'])['Description'].apply(lambda x: x.count()))

# Rename and reset Data Viz DF
df_top10_grouped.rename(columns={'Description':'Count'}, inplace=True)
df_top10_grouped.reset_index(inplace=True)

# Save as a Pickle file
df_top10_grouped.to_pickle('df_top10_grouped.pickle')